# 04 - Exploration SQL

Analyse exploratoire des données Silver avec SparkSQL.

## Configuration

In [ ]:
from config import get_s3_path, create_spark_session

SILVER_PATH = get_s3_path("silver", "flights")

spark = create_spark_session("ExplorationSQL")

df = spark.read.format("delta").load(SILVER_PATH)
df.createOrReplaceTempView("flights")

print(f"✅ {df.count():,} lignes chargées")

## Schéma des données

In [ ]:
df.printSchema()

## Statistiques par pays

In [ ]:
spark.sql("""
    SELECT 
        origin_country,
        COUNT(*) AS nb_observations,
        ROUND(AVG(altitude_meters), 2) AS avg_altitude,
        ROUND(AVG(velocity_kmh), 2) AS avg_velocity
    FROM flights
    WHERE origin_country IS NOT NULL
    GROUP BY origin_country
    ORDER BY nb_observations DESC
    LIMIT 10
""").show(truncate=False)

## Avions au sol vs en vol

In [ ]:
spark.sql("""
    SELECT 
        on_ground,
        COUNT(*) AS count,
        ROUND(AVG(velocity_kmh), 2) AS avg_velocity
    FROM flights
    GROUP BY on_ground
""").show()

## Distribution des altitudes

In [ ]:
spark.sql("""
    SELECT 
        CASE 
            WHEN altitude_meters IS NULL THEN 'NULL'
            WHEN altitude_meters < 1000 THEN '0-1000m'
            WHEN altitude_meters < 5000 THEN '1000-5000m'
            WHEN altitude_meters < 10000 THEN '5000-10000m'
            ELSE '10000m+'
        END AS altitude_range,
        COUNT(*) AS count
    FROM flights
    GROUP BY 1
    ORDER BY count DESC
""").show()

## Requête Batch avec Window Function

Analyse de l'évolution de chaque avion : altitude précédente, variation, et rang par vitesse dans son pays.

In [ ]:
spark.sql("""
    SELECT
        icao24,
        callsign,
        origin_country,
        event_timestamp,
        altitude_meters,
        LAG(altitude_meters) OVER (PARTITION BY icao24 ORDER BY event_timestamp) AS prev_altitude,
        altitude_meters - LAG(altitude_meters) OVER (PARTITION BY icao24 ORDER BY event_timestamp) AS altitude_change,
        ROUND(AVG(altitude_meters) OVER (PARTITION BY icao24 ORDER BY event_timestamp ROWS BETWEEN 5 PRECEDING AND CURRENT ROW), 2) AS rolling_avg_altitude,
        RANK() OVER (PARTITION BY origin_country ORDER BY velocity_kmh DESC) AS velocity_rank_in_country
    FROM flights
    WHERE icao24 IS NOT NULL AND altitude_meters IS NOT NULL
    ORDER BY icao24, event_timestamp
    LIMIT 20
""").show(truncate=False)

## Aperçu des données

In [ ]:
df.show(10, truncate=False)

## Visualisations avec Seaborn/Pandas

Analyses visuelles des données Silver converties en Pandas (échantillon pour limiter la mémoire).

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid")

# Chargement des données Silver_ML (contient flight_phase)
SILVER_ML_PATH = get_s3_path("silver", "flights_ml")
df_ml = spark.read.format("delta").load(SILVER_ML_PATH)

# Conversion Spark -> Pandas (échantillon de 10000 lignes)
pdf = df_ml.sample(fraction=0.1, seed=42).limit(10000).toPandas()
print(f"Echantillon: {len(pdf)} lignes depuis Silver_ML")

### 1. Distribution des altitudes (histogramme)

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=pdf, x="altitude_meters", bins=50, kde=True, color="steelblue")
plt.xlabel("Altitude (m)")
plt.ylabel("Nombre d'observations")
plt.title("Distribution des altitudes")
plt.tight_layout()
plt.show()

### 2. Répartition des phases de vol (pie chart)

In [ ]:
phase_counts = pdf["flight_phase"].value_counts()

plt.figure(figsize=(8, 8))
colors = sns.color_palette("Set2", len(phase_counts))
plt.pie(phase_counts, labels=phase_counts.index, autopct="%1.1f%%", colors=colors, startangle=90)
plt.title("Répartition des phases de vol")
plt.tight_layout()
plt.show()

### 3. Vitesse moyenne par pays (barplot horizontal)

In [ ]:
velocity_by_country = pdf.groupby("origin_country")["velocity_kmh"].mean().sort_values(ascending=True).tail(15)

plt.figure(figsize=(10, 8))
sns.barplot(x=velocity_by_country.values, y=velocity_by_country.index, palette="viridis")
plt.xlabel("Vitesse moyenne (km/h)")
plt.ylabel("Pays")
plt.title("Vitesse moyenne par pays (Top 15)")
plt.tight_layout()
plt.show()

### 4. Heatmap corrélation features ML

In [ ]:
# Sélection des colonnes numériques pour la corrélation
numeric_cols = ["altitude_meters", "velocity_kmh", "latitude", "longitude", 
                "altitude_change", "velocity_change", "rolling_avg_altitude"]
numeric_cols_available = [c for c in numeric_cols if c in pdf.columns]

corr_matrix = pdf[numeric_cols_available].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", center=0, fmt=".2f", square=True)
plt.title("Matrice de corrélation des features ML")
plt.tight_layout()
plt.show()

### 5. Évolution temporelle du trafic (lineplot)

In [ ]:
# Agrégation par minute pour l'évolution temporelle
pdf["event_timestamp"] = pd.to_datetime(pdf["event_timestamp"])
traffic_over_time = pdf.set_index("event_timestamp").resample("1min").size().reset_index(name="nb_flights")

plt.figure(figsize=(14, 6))
sns.lineplot(data=traffic_over_time, x="event_timestamp", y="nb_flights", color="coral")
plt.xlabel("Temps")
plt.ylabel("Nombre de vols")
plt.title("Évolution temporelle du trafic aérien")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()